In [1]:
class MyClass:
    pass

instance = MyClass()
type(instance)

__main__.MyClass

In [2]:
type(MyClass)

type

In [ ]:
class MyMeta(type):
    def __init__(self, ):

In [2]:
class ObjectCreator(object):
    pass

In [4]:
print(ObjectCreator)

<class '__main__.ObjectCreator'>


In [4]:
def echo(o): # 可以将类作为参数传递给函数
    print(o)

In [6]:
echo(ObjectCreator)

<class '__main__.ObjectCreator'>


In [3]:
print(hasattr(ObjectCreator, 'new_attribute'))

False


In [5]:
ObjectCreator.new_attribute = 'foo' # 你可以为类增加属性
print(hasattr(ObjectCreator, 'new_attribute'))

True


In [6]:
ObjectCreatorMirror = ObjectCreator # 你可以将类赋值给一个变量

In [7]:
print(ObjectCreatorMirror())

In [10]:
# 动态地创建类
def choose_class(name):
    if name == 'foo':
        class Foo(object):
            pass
        return Foo # 返回的是类，不是类的对象
    else:
        class Bar(object):
            pass
        return Bar

MyClass = choose_class('foo')
print(MyClass)

<class '__main__.choose_class.<locals>.Foo'>


In [11]:
print(MyClass()) # 也可以通过这个类创建实例，也就是对象

<__main__.choose_class.<locals>.Foo object at 0x7f7ee24f3a00>


这个还是不够动态，因为你仍需要自己编写整个类的代码。由于类也是对象，所以它们必须是通过什么东西来生成的才对。

type能动态的创建类，type可以接受一个类的描述作为参数，然后返回一个类。

type(类名，父类的元组（针对集成的情况，可以为空），包含属性的字段（名称和值）)

In [12]:
MyShinyClass = type('MyShinyClass', (), {}) # 返回一个类对象
print(MyShinyClass)

<class '__main__.MyShinyClass'>


In [13]:
print(MyShinyClass()) # 创建一个该类的实例

In [20]:
# type接受一个字典来为类定义属性，因此
class Foo(object):
    bar = True
# 等同于
print(Foo)

<class '__main__.Foo'>


In [26]:
Foo = type('Foo', (object, ), {'bar': True})

In [27]:
print(Foo)

<class '__main__.Foo'>


在Python中，类也是对象，你可以动态地创建类。这就是当你使用关键字class时Python在幕后做的事情，而这就是通过元类来实现的。

**元类**

元类就是创建类这种对象的东西。如果你喜欢的话，可以把元类称为“类工厂”。type就是Python内建元类，当然了，你也可以创建自己的元类。

***自定义元类***

元类的主要目的就是为了创建类时能够自动地改变类。

Python3中创建元类的语法：

# demo1

1. new是一个静态方法，而init是一个实例方法。
2. new方法会返回一个创建的实例，而init什么都不返回。
3. 只有在new返回一个cls的实例时后面的init才能被调用。
4. 当创建一个新的实例时调用new，初始化一个实例时用init。

In [29]:
# 记住，‘type’实际上是一个类，就像‘str’和‘int’一样
# 所以，你可以从type继承
class MetaA(type):
    # __new__是在__init__之前被调用的特殊方法
    # __new__是用来创建对象并返回之的方法
    # 而__init__只是用来将传入的参数初始化给对象
    # 你很少用到__new__，除非你希望能够控制对象的创建
    # 这里，创建的对象是类，我们希望能够自定义它，所以我们这里改写__new__
    # 如果你希望的话，你也可以在__init__中做些事情
    # 还有一些高级的用法会涉及到改写__call__特殊方法，但是我们这里不用
    def __new__(cls, name, bases, dct):
        print('MetaA.__new__')
        # 这种方式不会调用__init__方法
        # return type(name, base, dct)
        # 这种方式会调用_init__
        return type.__new__(cls, name, bases, dct)
    
    def __init__(cls, name, bases, dct):
        print('MetaA.__init__')

class A(object, metaclass=MetaA):
    pass

print(A())

MetaA.__new__
MetaA.__init__


# demo2

In [31]:
class ListMetaclass(type):
    
    """
        元类会自动将你通常传递给‘type’的参数作为自己的参数传入
        mcs：表示元类
        name：表示创建类的类名（在这里创建类就是继承Model类的子类User）
        bases：表示创建类继承的所有父类
        namespace：表示创建类的所有属性和方法（以键值对的字典形式）
    """
    def __new__(mcs, name, bases, namespace):
        namespace['add_item'] = lambda self, value: self.append(value)
        return type.__new__(mcs, name, bases, namespace)

# 通过metaclass，给该类动态添加了add方法
class MyList(list, metaclass=ListMetaclass):
    pass

l = MyList()
l.add_item(1)
l

[1]

## 究竟为什么要使用元类？

现在回到我们的大主题上来，究竟是为什么你会去使用这样一种容易出错且晦涩的特性？好吧，一般说来，你根本用不上它：

> "元类就是深度的魔法，99%的用户应该根本不必为此操心。如果你想搞清楚是否需要用到元类，那么你就不需要它。那些实际用到元类的人都非常清楚地知道他们需要做什么，而且根本不需要解释为什么要用元类。" --- Python界的领袖 Tim Peters

元类的主要用途是创建API。一个典型的例子是Django ORM。它允许你像这样定义

**一个简单的ORM demo**

In [43]:
class Field:
    def __init__(self, name, column_type):
        self.name = name
        self.column_type = column_type
    
    def __str__(self):
        return '<{}:{}>'.format(__class__.__name__, self.name)
    
class StringField(Field):
    def __init__(self, name):
        super().__init__(name, 'varchar(100)')
        
class IntegerField(Field):
    def __init__(self, name):
        super().__init__(name, 'bigint')

class ModelMetaclass(type):
    def __new__(mcs, name, bases, attrs):
        if name == 'Model':
            return type.__new__(mcs, name, bases, attrs)
        print('Found Model: {}'.format(name))
        mapping = dict()
        fields = list()
        # 将属性保存到mapping中
        for k, v in attrs.items():
            if isinstance(v, Field):
                print('Found mapping: {} ==> {}'.format(k, v))
                mapping[k] = v
                fields.append(k)
        # 将Model中的Field删除
        for k in mapping.keys():
            attrs.pop(k)
        
        attrs['__fields__'] = list(map(lambda f: '`{}`'.format(f), fields))
        attrs['__mapping__'] = mapping
        attrs['__table__'] = name
        return type.__new__(mcs, name, bases, attrs)

class Model(dict, metaclass=ModelMetaclass):
    def __init__(self, **kwargs):
        super().__init__(kwargs)
    
    def __getattr__(self, key):
        try:
            return self[key]
        except KeyError:
            raise AttributeError("'Model' object has no attribute '{}'".format(key))

    def __setattr__(self, key, value):
        self[key] = value
    
    def save(self):
        fields = []
        params = []
        args = []
        
        for k, v in self.__mapping__.items():
            print("{}------{}".format(k, v))
            params.append('?')
            args.append(getattr(self, k, None))
        sql = 'insert into {} ({}) values ({})'.format(self.__table__, ','.join(self.__fields__), ','.join(params))
        print('sql: {}'.format(sql))
        print('args: {}'.format(args))

class User(Model):
    id = IntegerField('id')
    name = StringField('name')
    email = StringField('email')
    password = StringField('password')

user = User(id=12345, name='Michael', email='test@orm.org', password='my-pwd')
user.save()

Found Model: User
Found mapping: id ==> <Field:id>
Found mapping: name ==> <Field:name>
Found mapping: email ==> <Field:email>
Found mapping: password ==> <Field:password>
id------<Field:id>
name------<Field:name>
email------<Field:email>
password------<Field:password>
sql: insert into User (`id`,`name`,`email`,`password`) values (?,?,?,?)
args: [12345, 'Michael', 'test@orm.org', 'my-pwd']
